In [5]:
import urllib.request as request
import urllib.parse as parse
from pprint import pprint
import json
import numpy as np
import xlsxwriter
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot

movie_titles = []
articles = []

class Crawler(object):
    def __init__(self, url, apikey, start_offset, end_offset):
        self.url = url
        self.apiKey = apikey
        self.startOffset = int(start_offset)
        self.endOffset = int(end_offset)

    def start(self):
        #articles = []
        #articles1 = []
        offsets = np.arange(self.startOffset, self.endOffset)
        
        workbook = xlsxwriter.Workbook('NYTimesMovies.xlsx')
        worksheet = workbook.add_worksheet()
        line = 1
        
        for offset in offsets:
        
            url = self.url + "/reviews/all.json"

            data = {}
            data['api-key'] = self.apiKey
            url_values = parse.urlencode(data)

            # add api key
            url = url + "?" + "offset=" + str(20*offset) + "&" + url_values
            with request.urlopen(url) as response:
                content = response.read()
                content = content.decode('utf-8')
                content = json.loads(content)
                for each in content['results']:
                    if "display_title" in each:
                        display_title = each['display_title']
                    else:
                        continue
                    movie_titles.append({
                        display_title
                    })
                
        print(len(movie_titles))
        with open('movie_titles.json', 'w') as f:
            for each in articles:
                json.dump(each, f)
                f.write('\n')
        
    def filter(self, path):
        filtered_articles = []
        with open(path, 'r') as f:
            for line in f:
                article = json.loads(line)
                if article['display_title'] is None:
                    continue
                else:
                    filtered_articles.append(article)
        
        print(len(filtered_articles))
        with open('newyorktimes_filtered.json', 'w') as f:
            for each in filtered_articles:
                json.dump(each, f)
                f.write('\n')
        


if __name__ == '__main__':
    crawler = Crawler(
        url = "https://api.nytimes.com/svc/movies/v2",
        apikey='5105d7c0e53347269a428d1c212e7de5',
        start_offset=0,
        end_offset=50
    )

crawler.start()
#crawler.filter('newyorktimes.json1')

1000


In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

data_sampley = Word2Vec(movie_titles, size=1, min_count=1)
data_sample = data_sampley[data_sampley.wv.vocab]
data_sample2 = set(data_sample.flatten())
data_sample3 = frozenset(data_sample2)
data_sample4 = data_sample.tolist()

movie_data = []
print(data_sample[0][0])
#for x in range(1000):
#    movie_data.append({
#        movie_titles[x],
#        data_sample3[x][0]
#    })
len(data_sample4)

0.4623673


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


998

In [35]:
from random import randint

user_ratings = [ ( randint(1, 20), k, randint(1, 10) ) for k in range(50,951) ]
ur_names = ['user_id','movie_id','rating']
ratings = pd.DataFrame.from_records(user_ratings,columns=ur_names)
user_profiles = [ ( k, randint(1, 2), randint(18, 22) ) for k in range(1,21) ]
up_names = ['user_id','sex','age']
users = pd.DataFrame.from_records(user_profiles,columns=up_names)
articles6 = [ ( k, movie_titles[k], data_sample4[k][0] ) for k in range(998) ]
movie_names = ['movie_id','movie_name','movie_data']
items = pd.DataFrame.from_records(articles6,columns=movie_names)

In [36]:
ratings_train = ratings
user_ratings2 = [ ( randint(1, 20), k, randint(1, 10) ) for k in range(1,50) ]
ratings_test = pd.DataFrame.from_records(user_ratings2,columns=ur_names)

In [37]:
ratings_train.shape, ratings_test.shape

((901, 3), (49, 3))

In [38]:
n_users = 20
n_items = 1000
data_matrix = np.zeros((n_users, n_items))
for line in ratings.itertuples():
    data_matrix[line[1]-1,line[2]-1] = line[3]

In [39]:
from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(data_matrix, metric='cosine')
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')

In [40]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [41]:
user_prediction = predict(data_matrix, user_similarity, type='user')
item_prediction = predict(data_matrix, item_similarity, type='item')

In [43]:
user_num = 9
movie_num = 4
gender = 'male'
if (user_profiles[user_num - 1][1] == 1):
    gender = 'male'
else:
    gender = 'female'
print('User ' + str(user_num) + " is a " + str(user_profiles[user_num - 1][2]) + " year old " + gender + " and has a " + str(user_prediction[user_num-1][movie_num] * 100) + ' percent chance of liking ' + str(movie_titles[movie_num]) + ' according to model 1')
print('User ' + str(user_num) + " is a " + str(user_profiles[user_num - 1][2]) + " year old " + gender + " and has a " + str(item_prediction[user_num-1][movie_num] * 100) + ' percent chance of liking ' + str(movie_titles[movie_num]) + ' according to model 2')

User 9 is a 19 year old female and has a 3.5684210526315763 percent chance of liking {'The House That Jack Built'} according to model 1
User 9 is a 19 year old female and has a 28.52852852852853 percent chance of liking {'The House That Jack Built'} according to model 2
